In [3]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *

StatementMeta(, ee8fd62c-c77e-40a6-b912-0fc26360b97c, 5, Finished, Available)

In [4]:
%%sql
create table if not exists TMDW_FinalProj.gld_shipmode
(
    SM_ID	Long	,
Ship_Mode	string	,
Created_TS	timestamp,
Modified_TS timestamp
)

StatementMeta(, ee8fd62c-c77e-40a6-b912-0fc26360b97c, 6, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [5]:
Maximum_Date=spark.sql("select coalesce(max(Modified_TS),'1900-01-01') from TMDW_FinalProj.gld_shipmode").first()[0]

StatementMeta(, ee8fd62c-c77e-40a6-b912-0fc26360b97c, 7, Finished, Available)

In [6]:
Maximum_Date

StatementMeta(, ee8fd62c-c77e-40a6-b912-0fc26360b97c, 8, Finished, Available)

'2024-05-04 23:42:49.463817'

In [8]:
dataframe_bze=spark.sql("select distinct Ship_Mode from TMDW_FinalProj.bronzesales where Modified_TS>'{}'".format(Maximum_Date))

StatementMeta(, ee8fd62c-c77e-40a6-b912-0fc26360b97c, 10, Finished, Available)

In [9]:
dataframe_bze.show()

StatementMeta(, ee8fd62c-c77e-40a6-b912-0fc26360b97c, 11, Finished, Available)

+---------+
|Ship_Mode|
+---------+
+---------+



In [11]:
Maximum_ID=spark.sql("select coalesce(max(SM_ID),0) from TMDW_FinalProj.gld_shipmode").first()[0]

StatementMeta(, ee8fd62c-c77e-40a6-b912-0fc26360b97c, 13, Finished, Available)

In [12]:
dataframe_final=dataframe_bze.withColumn("SM_ID",monotonically_increasing_id()+Maximum_ID+1)
dataframe_final.createOrReplaceTempView("View_ShipMode")

StatementMeta(, ee8fd62c-c77e-40a6-b912-0fc26360b97c, 14, Finished, Available)

In [13]:
%%sql
merge into TMDW_FinalProj.gld_shipmode as gsm 
using View_ShipMode as vsm  
on gsm.Ship_Mode=vsm.Ship_Mode
when MATCHED then 
update set
Modified_TS= current_timestamp()

when not matched then
INSERT
(
gsm.SM_ID,
gsm.Ship_Mode,
gsm.Created_TS,
gsm.Modified_TS
)
VALUES
(
vsm.SM_ID,
vsm.Ship_Mode,
current_timestamp(),
current_timestamp()

)

StatementMeta(, ee8fd62c-c77e-40a6-b912-0fc26360b97c, 15, Finished, Available)

<Spark SQL result set with 1 rows and 4 fields>